In [1]:
import os
import pandas as pd
import numpy as np
import glob
import datetime
from datetime import datetime
import bz2
import re
import glob
from tqdm import tqdm
import random
from datetime import timedelta

# This mutes useless warnings:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
from data_loader import NASAAirportDataset

# this will take a while
train_ds = NASAAirportDataset('KCLT', 'data/preprocess/train')



In [2]:

# Change these times and samples
start = datetime(2022, 9, 30, 0, 0, 0)  # Start of the range
end = datetime(2023, 9, 30, 23, 0)  # End of the range
n_samples = 10
time_samples = [start + timedelta(seconds=random.randint(0, int((end - start).total_seconds()))) for _ in range(n_samples)]
print(time_samples)


[datetime.datetime(2022, 12, 29, 10, 58, 22), datetime.datetime(2022, 12, 26, 14, 5, 54), datetime.datetime(2022, 12, 12, 19, 10, 41), datetime.datetime(2023, 8, 9, 3, 1, 50), datetime.datetime(2023, 8, 30, 21, 33, 16), datetime.datetime(2023, 4, 16, 19, 38, 58), datetime.datetime(2022, 10, 21, 16, 7, 22), datetime.datetime(2023, 6, 10, 8, 26, 1), datetime.datetime(2023, 8, 18, 1, 32, 28), datetime.datetime(2023, 1, 11, 10, 1, 3)]


In [15]:
import torch

for i in time_samples:
    print('Flights at time', i)
    inputs, targets =train_ds[i]
    print(inputs)
    print(targets)
    # for flight_data in train_ds[i]:
    #     train model on one row of (flight_data)



Flights at time 2023-07-25 01:37:27


/home/miner/repos/523Proj/data_loader.py:603: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  runway_rows = runway_rows.join(self.tbfm_df, on='gufi').groupby(level='gufi').agg(max)
/home/miner/repos/523Proj/data_loader.py:604: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  runway_rows = runway_rows.join(self.tfm_df, on='gufi').groupby(level='gufi').agg(max)
/home/miner/repos/523Proj/data_loader.py:617: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the

Empty DataFrame
Columns: [arrival_runway_actual_time, arrival_runway_actual_18C, arrival_runway_actual_18L, arrival_runway_actual_18R, arrival_runway_actual_23, arrival_runway_actual_36C, arrival_runway_actual_36L, arrival_runway_actual_36R, arrival_runway_actual_5, arrival_runway_actual_nan, aircraft_engine_class_JET, aircraft_engine_class_PISTON, aircraft_engine_class_TURBO, aircraft_engine_class_nan, aircraft_type_A124, aircraft_type_A139, aircraft_type_A20N, aircraft_type_A21N, aircraft_type_A306, aircraft_type_A319, aircraft_type_A320, aircraft_type_A321, aircraft_type_A332, aircraft_type_A333, aircraft_type_A339, aircraft_type_A345, aircraft_type_A346, aircraft_type_A359, aircraft_type_A35K, aircraft_type_AA5, aircraft_type_AC11, aircraft_type_AC50, aircraft_type_AC90, aircraft_type_AC95, aircraft_type_AEST, aircraft_type_AS50, aircraft_type_ASTR, aircraft_type_B190, aircraft_type_B350, aircraft_type_B38M, aircraft_type_B39M, aircraft_type_B412, aircraft_type_B429, aircraft_type_

/home/miner/repos/523Proj/data_loader.py:603: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  runway_rows = runway_rows.join(self.tbfm_df, on='gufi').groupby(level='gufi').agg(max)
/home/miner/repos/523Proj/data_loader.py:604: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  runway_rows = runway_rows.join(self.tfm_df, on='gufi').groupby(level='gufi').agg(max)
/home/miner/repos/523Proj/data_loader.py:617: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the

                                      arrival_runway_actual_time  \
gufi                                                               
AAL1059.BNA.CLT.230715.2110.0135.TFM                    0.791574   
AAL1157.CUN.CLT.230715.1740.0171.TFM                    0.145833   
AAL1379.BDL.CLT.230715.1900.0249.TFM                    0.483056   
AAL1423.BOS.CLT.230715.1720.0130.TFM                    0.520463   
AAL1464.BWI.CLT.230715.2105.0039.TFM                    0.940463   
...                                                          ...   
UAL609.ORD.CLT.230715.1900.0202.TFM                     0.506944   
UAL758.EWR.CLT.230715.2025.0156.TFM                     0.998333   
VTE3757.PAH.CLT.230715.1915.0135.TFM                    0.299352   
VTE3853.LWB.CLT.230715.1935.0058.TFM                    0.193704   
VTE3855.LWB.CLT.230715.2115.0032.TFM                    0.721204   

                                      arrival_runway_actual_18C  \
gufi                                            

...

/home/miner/repos/523Proj/data_loader.py:603: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  runway_rows = runway_rows.join(self.tbfm_df, on='gufi').groupby(level='gufi').agg(max)
/home/miner/repos/523Proj/data_loader.py:604: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  runway_rows = runway_rows.join(self.tfm_df, on='gufi').groupby(level='gufi').agg(max)
/home/miner/repos/523Proj/data_loader.py:617: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the

Empty DataFrame
Columns: [arrival_runway_actual_time, arrival_runway_actual_18C, arrival_runway_actual_18L, arrival_runway_actual_18R, arrival_runway_actual_23, arrival_runway_actual_36C, arrival_runway_actual_36L, arrival_runway_actual_36R, arrival_runway_actual_5, arrival_runway_actual_nan, aircraft_engine_class_JET, aircraft_engine_class_PISTON, aircraft_engine_class_TURBO, aircraft_engine_class_nan, aircraft_type_A124, aircraft_type_A139, aircraft_type_A20N, aircraft_type_A21N, aircraft_type_A306, aircraft_type_A319, aircraft_type_A320, aircraft_type_A321, aircraft_type_A332, aircraft_type_A333, aircraft_type_A339, aircraft_type_A345, aircraft_type_A346, aircraft_type_A359, aircraft_type_A35K, aircraft_type_AA5, aircraft_type_AC11, aircraft_type_AC50, aircraft_type_AC90, aircraft_type_AC95, aircraft_type_AEST, aircraft_type_AS50, aircraft_type_ASTR, aircraft_type_B190, aircraft_type_B350, aircraft_type_B38M, aircraft_type_B39M, aircraft_type_B412, aircraft_type_B429, aircraft_type_

/home/miner/repos/523Proj/data_loader.py:603: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  runway_rows = runway_rows.join(self.tbfm_df, on='gufi').groupby(level='gufi').agg(max)
/home/miner/repos/523Proj/data_loader.py:604: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  runway_rows = runway_rows.join(self.tfm_df, on='gufi').groupby(level='gufi').agg(max)
/home/miner/repos/523Proj/data_loader.py:617: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the

                                      arrival_runway_actual_time  \
gufi                                                               
AAL1022.MCO.CLT.221125.2230.0229.TFM                    0.680926   
AAL1060.SXM.CLT.221125.1910.0121.TFM                    0.367130   
AAL1148.STT.CLT.221125.2110.0115.TFM                    0.919167   
AAL1195.STT.CLT.221125.2005.0114.TFM                    0.606759   
AAL1216.EWR.CLT.221125.2225.0058.TFM                    0.670000   
...                                                          ...   
PJC83.ISM.CLT.221126.0159.0001.TFM                      0.438148   
RPA5657.LGA.CLT.221125.2255.0021.TFM                    0.838519   
SWA2495.MDW.CLT.221125.2110.0092.TFM                    0.174815   
UAL2544.DEN.CLT.221125.1805.0146.TFM                    0.069907   
VTE3855.LWB.CLT.221125.2155.0135.TFM                    0.241481   

                                      arrival_runway_actual_18C  \
gufi                                            

/home/miner/repos/523Proj/data_loader.py:603: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  runway_rows = runway_rows.join(self.tbfm_df, on='gufi').groupby(level='gufi').agg(max)
/home/miner/repos/523Proj/data_loader.py:604: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  runway_rows = runway_rows.join(self.tfm_df, on='gufi').groupby(level='gufi').agg(max)
/home/miner/repos/523Proj/data_loader.py:617: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the